## Continuation of EDA and Evaluating Valuation Set on German Traffic Sign Dataset using EfficientNet and ResNet18
### Billy Ryan

This file is a continuation of the initial eda file, just with an update of validation set to see the increase in accuracy. Everything else is the same but I thought it was worth to see how the project evolved over time so I decided to keep both files separate as a good comparison tool. All of the EDA conducted remains the same.

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import kagglehub
from typing import List
import random
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import math
from typing import List, Tuple
from collections import Counter
import seaborn as sns

In [ ]:
# Download latest version of the dataset
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

In [ ]:
print(os.listdir(path))

train_df_path =  os.path.join(path, "Train.csv")
test_df_path =  os.path.join(path, "Test.csv")

train_img_path =  os.path.join(path, "Train")
test_img_path =  os.path.join(path, "Test")

Much of the following EDA was lifted and/or inspired from Neva's and Gracie's work alongside many kaggle notebooks which have been referenced below, making alterations for ease of use and more thorough analysis. This file will not go into much detail as that has been done separately either in the report folder or in a markdown file in my (Billy) folder.

In [ ]:
train_df = pd.read_csv(train_df_path)
test_df = pd.read_csv(test_df_path)

In [ ]:
train_df['ClassId'].value_counts()

In [ ]:
counts = train_df['ClassId'].value_counts().sort_index()

plt.figure(figsize=(15,5))
plt.bar(counts.index, counts.values)
plt.title("Distribution of images per class")
plt.xlabel("Class")
plt.ylabel("Number of images")
plt.xticks(counts.index)
plt.show()

In [ ]:
class_names = {
    0:"Speed Limit (20Km/hr)", 1:"Speed Limit (30Km/hr)", 
    2:"Speed Limit (50Km/hr)", 3: "Speed Limit (60Km/hr)", 
    4: "Speed Limit (70Km/hr)", 5: "Speed Limit (80Km/hr)",
    6: "End of Speed Limit (80Km/hr)", 7: "Speed Limit (100Km/hr)", 
    8: "Speed Limit (120Km/hr)", 9: "No Passing", 
    10: "No Passing for trucks over 3.5 tons", 11: "Right of way", 
    12: "Priotity Road", 13: "Yeild right of way",
    14: "Stop", 15: "Prohibited for all vehicles",
    16: "Trucks and tractors over 3.5 tons prohibited", 17: "Entery prohibited",
    18: "Danger", 19: "Single curve left",
    20: "Single curve right", 21: "Double curve",
    22: "Rough road", 23: "Slippery road",
    24: "Road narrows", 25: "Construction side ahead",
    26: "Signal lights ahead", 27: "Pedestrian crosswalk ahead",
    28: "Children", 29: "Bicycle crossing",
    30: "Unexpected ice danger", 31: "Wild animal crossing",
    32: "End of restrection", 33: "Mandatory direction of travel right",
    34: "Mandatory direction of travel left", 35: "Mandatory direction of travel ahead",
    36: "Straight or right", 37: "Straight or left",
    38: "Keep right", 39: "Keep left",
    40: "Traffic circle", 41: "End of no passing zone cars",
    42: "End of no passing zone vehicle over 3.5 tons"
}

train_df["ClassName"] = train_df['ClassId'].map(class_names)
test_df["ClassName"] = test_df['ClassId'].map(class_names)

In [ ]:
print(f"Maximum images per class: {train_df['ClassName'].value_counts().max()} (Class: {train_df['ClassName'].value_counts().idxmax()})")
print(f"Minimum images per class: {train_df['ClassName'].value_counts().min()} (Class: {train_df['ClassName'].value_counts().idxmin()})")
print(f"Average images per class: {train_df['ClassName'].value_counts().mean():.1f}")

In [ ]:
plt.figure(figsize=(16, 8))
ax = sns.countplot(data=train_df, x="ClassName", order=train_df["ClassName"].value_counts().index, color="#5F98E2")
plt.xlabel("Class Name")
plt.ylabel("Count")
plt.xticks(rotation=90)
for container in ax.containers:
    ax.bar_label(container, fontsize=7)
plt.show()

In [ ]:
def random_image_generator(class_id):
    if class_id < 0 or class_id > 42:
        raise ValueError("class_id must be between 0 and 42 inclusive.")

    folder = os.path.join(train_img_path, str(class_id))
    image_files = [f for f in os.listdir(folder) if not f.startswith(".")]

    filename = random.choice(image_files)
    sample_path = os.path.join(folder, filename)

    img = Image.open(sample_path)

    return img

Now we can randomise images that are in our training dataset. This will help familiarise the user with the types of images we are dealing with.

In [ ]:
random_class = random.randint(0, 42)
img1 = random_image_generator(random_class)

plt.figure(figsize=(8,8))
plt.imshow(img1)
plt.axis("off")
plt.show()

In [ ]:
def get_image_paths(filepath: str) -> Tuple[List[str], List[str]]:
    image_paths = []
    class_labels = []

    for root, _, files in os.walk(filepath):
        for filename in files:
            if filename.lower().endswith((".png", ".jpg", ".jpeg", ".ppm", ".bmp")):
                full_path = os.path.join(root, filename)
                image_paths.append(full_path)

                # grab class folder name
                class_id = os.path.basename(root)
                class_labels.append(class_id)

    return image_paths, class_labels

train_image_paths, train_labels = get_image_paths(train_img_path)

In [ ]:
def print_sample_images(n, cols):
    total = len(train_image_paths)
    num_images = min(n*cols, total)

    if num_images >= total:
        print("WARNING: Total images exceed available images. Returning all available images. This may take a while.")

    if (n, cols) >= (100, 5):
        print("WARNING: You are attempting to plot many images. This may take a while.")

    plt.figure(figsize=(cols*3, n*3))

    for i, k in enumerate(random.sample(range(total), num_images), start=1):
        img = plt.imread(train_image_paths[k])
        plt.subplot(n, cols, i)
        plt.imshow(img)
        plt.title(class_names[int(train_labels[k])])
        plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
print_sample_images(6,7)

In [ ]:
def random_sample_images_per_class():
    for c in sorted(train_df['ClassId'].unique()):
        folder = os.path.join(train_img_path, str(c))
        filenames = os.listdir(folder)
        
        sample_files = random.sample(filenames, min(5, len(filenames)))
        
        # Plot images
        plt.figure(figsize=(15, 3))
        class_name = class_names.get(c, f"Class {c}")
        plt.suptitle(f"{class_name} (Class {c})", fontsize=16)
        
        for i, f in enumerate(sample_files):
            img_path = os.path.join(folder, f)
            img = Image.open(img_path)
            
            plt.subplot(1, 5, i+1)
            plt.imshow(img)
            plt.axis("off")
        
        plt.show()

In [ ]:
random_sample_images_per_class()

In [ ]:
def resolutions(image_paths, max_images=None):
    resolutions = []
    for path in image_paths[:max_images]:
        with Image.open(path) as img:
            resolutions.append(img.size)
    return resolutions

resolutions(train_image_paths, max_images=5000)

In [ ]:
def reso_counter(image_paths, max_images=None):
    res_counter = Counter(resolutions(image_paths, max_images))
    print("Top resolutions (Width x Height : Count):")
    for (w, h), count in res_counter.most_common(10):
        print(f"{w} x {h} : {count}")

reso_counter(train_image_paths, max_images=5000)

In [ ]:
def reso_histogram(image_paths, max_images=None):
    widths, heights = zip(*resolutions(image_paths, max_images))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.hist(widths, bins=20, color="#5F98E2")
    ax1.set(xlabel="Width (pixels)", ylabel="Count", title="Image widths")
    ax2.hist(heights, bins=20, color="#5F98E2")
    ax2.set(xlabel="Height (pixels)", ylabel="Count", title="Image heights")
    plt.tight_layout()
    plt.show()

    return widths, heights

widths, heights = reso_histogram(train_image_paths, max_images=5000)

In [ ]:
img = np.array(Image.open(train_image_paths[0])) 
plt.hist(img.ravel(), bins=255)
plt.title("Pixel intensity distribution")
plt.show()

In [ ]:
def random_pixel_values():
    random.seed(42)
    pixel_values = []

    # take up to 10 random images from each class
    for c in sorted(train_df['ClassId'].unique()):
        folder = os.path.join(train_img_path, str(c))
        filenames = os.listdir(folder)
        sample_files = random.sample(filenames, min(10, len(filenames)))
        
        for f in sample_files:
            img = Image.open(os.path.join(folder, f)).convert('RGB')
            img_array = np.array(img)
            pixel_values.extend(img_array.flatten())

    return pixel_values

In [ ]:
# Plot histogram
plt.figure(figsize=(10,5))
plt.hist(random_pixel_values(), bins=50, color='gray')
plt.title("Pixel Intensity Distribution Across Dataset")
plt.xlabel("Pixel Value (0-255)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
pixels = np.array(random_pixel_values())

# Compute more statistics
print("Min pixel value:", np.min(pixels))
print("25th percentile:", np.percentile(pixels, 25))
print("Median (50th percentile):", np.median(pixels))
print("75th percentile:", np.percentile(pixels, 75))
print("Max pixel value:", np.max(pixels))
print("Mean pixel value:", np.mean(pixels))
print("Standard deviation:", np.std(pixels))

In [ ]:
random.seed(123)
np.random.seed(123)

def augment_image(img):
    # Rotation ±10°
    angle = random.uniform(-10, 10)
    img = img.rotate(angle)

    # Random horizontal and vertical shift (10% max)
    max_shift_x = int(0.1 * img.width)
    max_shift_y = int(0.1 * img.height)
    shift_x = random.randint(-max_shift_x, max_shift_x)
    shift_y = random.randint(-max_shift_y, max_shift_y)
    img = img.transform(img.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))

    # Random zoom/crop ±10%
    zoom_factor = random.uniform(0.9, 1.1)
    w, h = img.size
    new_w, new_h = int(w * zoom_factor), int(h * zoom_factor)
    img = img.resize((new_w, new_h), Image.BILINEAR)
    img = img.crop((0, 0, w, h))  # crop or pad back to original size

    # Brightness adjustment ±20%
    enhancer = ImageEnhance.Brightness(img)
    factor = random.uniform(0.8, 1.2)
    img = enhancer.enhance(factor)

    return img

In [ ]:
image_number = random.randint(0, 42)

img1 = random_image_generator(image_number)

aug_img1 = augment_image(img1)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow(img1)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(aug_img1)
plt.title("Augmented")
plt.axis("off")
plt.show()

In [ ]:
classes = sorted(os.listdir(train_img_path))
print("Number of classes:", len(classes))

image_count = {}
half_dataset = []  # final list of selected images

for c in classes:
    folder = os.path.join(train_img_path, c)
    files = os.listdir(folder)

    # count images like your EDA
    image_count[c] = len(files)

    # take 50% of the images in this class
    half = len(files) // 2
    selected = np.random.choice(files, half, replace=False)

    # store full paths
    for f in selected:
        half_dataset.append(os.path.join(folder, f))

print("Original total images:", sum(image_count.values()))
print("Reduced total images:", len(half_dataset))

In [ ]:
from PIL import Image
from torch.utils.data import Dataset

class FilepathDataset(Dataset):
    def __init__(self, filepaths, transform=None):
        self.filepaths = filepaths
        self.transform = transform

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        path = self.filepaths[idx]
        img = Image.open(path).convert("RGB")

        # label = folder name → class index
        label = int(os.path.basename(os.path.dirname(path)))

        if self.transform:
            img = self.transform(img)
        else:
            from torchvision import transforms
            img = transforms.ToTensor()(img)

        return img, label


In [ ]:
import torch
from torchvision import transforms
from torchvision.models import EfficientNet_B0_Weights
IMG_SIZE = 224
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

#train transform and this includes Nevas EDA findings 

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # resize all images to 224×224
    transforms.RandomRotation(20),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2), 
    transforms.ToTensor(),           # convert PIL image -> tensor
    transforms.Normalize(imagenet_mean, imagenet_std)
])

#validation test transform (no augmentation)
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

In [ ]:
SEED = 123
full_ds = FilepathDataset(half_dataset)

train_size = int(0.8 * len(half_dataset))
val_size = len(full_ds) - train_size

train_ds, val_ds = torch.utils.data.random_split(
    full_ds,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(SEED)
)
train_ds.dataset.transform = train_transforms
val_ds.dataset.transform = val_transforms

In [ ]:
from torch.utils.data import DataLoader 
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

See that train dataset has been reduced to 80% of inital length, as we apportioned the remaining 20% to validation set.

Will follow up with evaluation, importing Neva and Gracie's Models

In [ ]:
os.getcwd()
os.chdir("C:/Users/billy/DataScienceToolbox-Project2")
print(os.listdir())

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

NUM_CLASSES = 43

# Load EfficientNet base model
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
base_model = efficientnet_b0(weights=weights)

# Remove classifier → keep global avg pooling only
base_model.classifier = nn.Identity()

# Freeze base model
for param in base_model.parameters():
    param.requires_grad = False

# Build the same custom head Neva used
model = nn.Sequential(
    base_model,                # (0)
    nn.Linear(1280, 256),      # (2)
    nn.ReLU(),                 # (3)
    nn.Dropout(0.4),           # (4)
    nn.Linear(256, NUM_CLASSES),  # (5)
    nn.Softmax(dim=1)          # (6)
)

# Load the saved weights
state_dict = torch.load("Neva/efficientnet_best.pth", map_location="cpu")
model.load_state_dict(state_dict)
model.eval()

In [ ]:
from torchvision.models import ResNet50_Weights
from torchvision import models

model_resnet18 = models.resnet18(weights=None)
model_resnet18.fc = torch.nn.Linear(512, 43)   # example for 43 traffic signs
state_dict_resnet18 = torch.load("Gracie/resnet18_traffic_signs.pth", map_location="cpu")
model_resnet18.load_state_dict(state_dict_resnet18)
model_resnet18.eval()

below is validation test.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model_resnet18.to(device)

correct = 0
correct_resnet18 = 0
total = 0

criterion = nn.CrossEntropyLoss()  # only if you want loss

model.eval()
model_resnet18.eval()
with torch.no_grad():  # disables gradient calculation
    for images, labels in val_loader:   # your validation DataLoader
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        outputs_resnet18 = model_resnet18(images)
        _, predicted_resnet18 = torch.max(outputs_resnet18, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        correct_resnet18 += (predicted_resnet18 == labels).sum().item()

accuracy = 100 * correct / total
accuracy_resnet18 = 100 * correct_resnet18 / total
print(f"Validation Accuracy for Efficient Net: {accuracy:.2f}%")
print(f"Validation Accuracy for ResNet18: {accuracy_resnet18:.2f}%")

In [ ]:
class_names = [class_names[i] for i in range(NUM_CLASSES)]

In [ ]:
from sklearn.metrics import classification_report

def classification_report_model(model, val_loader, device):
    model.to(device)    
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    report = classification_report(all_labels, all_preds, zero_division=0)
    print(report)
    return all_labels, all_preds

In [ ]:
from sklearn.metrics import confusion_matrix
true_effnet, pred_effnet = classification_report_model(model, val_loader, device)
cm = confusion_matrix(true_effnet, pred_effnet)

In [ ]:
true_resnet, pred_resnet = classification_report_model(model_resnet18, val_loader, device)
cm_resnet = confusion_matrix(true_resnet, pred_resnet)

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=False, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix for Efficient Net Model")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(cm_resnet, annot=False, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix for ResNet18 Model")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def show_misclassified(model, loader, device, class_names, max_images=16):
    model.eval()
    images_list, true_list, pred_list = [], [], []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = outputs.max(1)

            mismatch = preds != labels
            if mismatch.any():
                for img, t, p, m in zip(images, labels, preds, mismatch):
                    if m:
                        images_list.append(img.cpu())
                        true_list.append(t.item())
                        pred_list.append(p.item())
                    if len(images_list) >= max_images:
                        break
            if len(images_list) >= max_images:
                break

    
    n = len(images_list)
    cols = 4
    rows = (n + cols - 1) // cols
    plt.figure(figsize=(3*cols, 3*rows))

    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std  = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

    for i in range(n):
        img = images_list[i]        
        img = img * std + mean       
        img = img.permute(1, 2, 0)
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"T: {class_names[true_list[i]]}\nP: {class_names[pred_list[i]]}")
    plt.tight_layout()
    plt.show()

# lets test it
show_misclassified(model_resnet18, val_loader, device, class_names)

In [ ]:
show_misclassified(model, val_loader, device, class_names)

Took commands from Neva and Gracie's and attempted to configure them for both models. All of this evaluation so far has been on the validation set and has allowed us to gain a good perspective on how these models should perform on the full dataset. We now test the models on the unseen data. See eval file.